In [2]:
import pickle

with open("cbr_lightning/tokenizer.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))


<class 'tokenizer.WordTokenizer'>


In [9]:
from word_tok import WordTokenizer
import datasets

# Load raw data
raw = datasets.load_from_disk("cbr_lightning/wikitext-103-local")
train_texts = [t for t in raw['train']['text'] if t.strip()]

# Build tokenizer
tokenizer = WordTokenizer(vocab_size=50000, min_freq=2)
tokenizer.build_vocab(train_texts)

# Save tokenizer
tokenizer.save("tokenizer.pkl")



Vocabulary built with 49999 tokens.
Tokenizer saved to tokenizer.pkl


In [6]:
filepath = './tokenizer.pkl'
with open(filepath, 'rb') as f:
        data = pickle.load(f)

In [10]:
from word_tok import WordTokenizer  # your local tokenizer.py

tokenizer = WordTokenizer()
tokenizer.load(filepath)

Tokenizer loaded from ./tokenizer.pkl


In [11]:
import datasets
raw = datasets.load_from_disk("cbr_lightning/wikitext-103-local")
print(raw)
print(sum(len(t.split()) for t in raw["train"]["text"]))


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})
101425671
